In [1]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

a = cobra.io.read_sbml_model("iEC1344_C.xml")

Academic license - for non-commercial use only - expires 2023-02-21
Using license file /home/schwartzlab/gurobi.lic


In [2]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    i3= 101

    while i3>100 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<100:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values

In [3]:
i= 0
diff_saturated= list()
diff_indecr= list()
flat= list()
r = a.reactions[0]


for r in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 1:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.02 and d.iloc[n2,  0] < max(d["Biomass"])+0.02:
                count= count+1
    
            n2= n2+1
        
        if count> 0.8*len(d) and count< 0.95*len(d):
            diff_saturated.append(r_id)

        elif count> 0.95*len(d):
            flat.append(r_id)
        
        else:
            diff_indecr.append(r_id)

    
    i= i+1

flat
diff_indecr
diff_saturated

['EX_h_e',
 'EX_lys__L_e',
 'EX_minohp_e',
 'EX_ppt_e',
 'EX_fe2_e',
 'EX_fe3_e',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'ACACT8r_copy1',
 'ADA',
 'ADD',
 'DADA',
 'CTECOAI7_copy1',
 'CTECOAI8_copy1',
 'DHAPT',
 'DAAD',
 'EAR120y',
 'EAR140y',
 'EAR160y',
 'EAR161y',
 'EAR180y',
 'EAR181y',
 'FADRx2',
 'FDH4pp',
 'ASNS1',
 'FEROpp',
 'FE2tex',
 'FLVR',
 'FMNRx2',
 'FE3tex',
 'GLUt2rpp',
 'GLYCK',
 'GLYCK2',
 'GLYCLTDy',
 'Htex',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8',
 'GLYCTO2',
 'GLYOX3',
 'L_LACD2_copy1',
 'L_LACD3_copy1',
 'MMM',
 'MICITDr_copy1',
 'NTD8',
 'NTPTP1',
 'GTPDPDP',
 'NTRIR4pp',
 'SSALx',
 'SUCOAS',
 'SERD_D',
 'SERD_L',
 'THRA',
 'THRD',
 'TRPS2',
 'TRSARr',
 'XTSNH_copy1',
 'L_LACD3_copy2',
 'MMM2',
 'L_LACD2_copy2',
 'XTSNH_copy2']

In [4]:
flat_only_above_zero= list()
diff_indecr_only_above_zero= list()
diff_saturated_only_above_zero= list()

i=0
r_id = l2[i]

while i< len(l2):    #finds reactions with an increasing/decreasing pattern above zero
    r_id = l2[i]
    if r_id in diff_indecr and r_id not in l3 and r_id not in diff_indecr_only_above_zero:
        diff_indecr_only_above_zero.append(r_id)
    i= i + 1

diff_indecr_only_above_zero

i=0
r_id = l2[i]

while i< len(l2):   #finds reactions with a flat pattern above zero
    r_id = l2[i]
    if r_id in flat and r_id not in l3 and r_id not in flat_only_above_zero:
        flat_only_above_zero.append(r_id)
    i= i + 1

flat_only_above_zero

i=0
r_id = l2[i]

while i< len(l2):  #finds reactions with a saturating pattern above zero
    r_id = l2[i]
    if r_id in diff_saturated and r_id not in l3 and r_id not in diff_saturated_only_above_zero:
        diff_saturated_only_above_zero.append(r_id)
    i= i + 1

diff_saturated_only_above_zero

[]

In [5]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[100,  2])
    
    if v == 1 and csvFile.iloc[100,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[100,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])

['LCTStex']
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_met__L_e', 'EX_metsox_R__L_e', 'EX_metsox_S__L_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16

In [6]:
flat_only_below_zero= list()
diff_indecr_only_below_zero= list()
diff_saturated_only_below_zero= list()

i=0
r_id = l3[i]

while i< len(l3):    #finds reactions with an increasing/decreasing pattern below zero
    r_id = l3[i]
    if r_id in diff_indecr and r_id not in l2 and r_id not in diff_indecr_only_below_zero:
        diff_indecr_only_below_zero.append(r_id)
    i= i + 1

diff_indecr_only_below_zero

i=0
r_id = l3[i]

while i< len(l3):   #finds reactions with a flat pattern below zero
    r_id = l3[i]
    if r_id in flat and r_id not in l2 and r_id not in flat_only_below_zero:
        flat_only_below_zero.append(r_id)
    i= i + 1

flat_only_below_zero

i=0
r_id = l3[i]

while i< len(l3):   #finds reactions with a saturating pattern below zero
    r_id = l3[i]
    if r_id in diff_saturated and r_id not in l2 and r_id not in diff_saturated_only_below_zero:
        diff_saturated_only_below_zero.append(r_id)
    i= i + 1

diff_saturated_only_below_zero

['EX_minohp_e', 'EX_ppt_e', 'NTRIR4pp']

In [7]:
set1= set(flat)
set2= set(flat_only_above_zero)
set3= set(flat_only_below_zero)
flat_both_positive_and_negative= list(set1-set2-set3)   #finds reactions that are flat for both above and below 0 flux values
flat_both_positive_and_negative

['CRNt8pp',
 'NAt3pp',
 'EAR40y',
 'HACD3',
 'EAR60y',
 'EAR141y',
 'DHORDfum',
 'GLYCLTt4pp',
 'HACD1i',
 'ACACT8',
 'ACS',
 'GLYt4pp',
 'HACD6i',
 'VPAMTr_copy1',
 'ACACT4r',
 'CTECOAI6_copy1',
 'MDDCP3pp',
 'ALAt2pp_copy1',
 'ICHORS_copy1',
 'SERt4pp',
 'MICITDr_copy2',
 'ADK3',
 'ACACT9',
 'KAT1',
 'PROt4pp',
 'GLCP2',
 'MLDCP2App',
 'EAR80y',
 'HACD2i',
 'ACACT13',
 'ASPt2pp_copy1',
 'ACACT2r',
 'L_LACtex_copy1',
 'NDPK1',
 'ACACT1r',
 'VALTA',
 'ALDD19xr_copy2',
 'ACt4pp',
 'CTECOAI7_copy2',
 'HACD4i',
 'GLCtex_copy2',
 'ALAt4pp',
 'ADK1',
 'ABUTt2pp',
 'HACD5i',
 'HACD4',
 'DARBt2rpp_copy2',
 'EAR100y',
 'ACACT3r',
 'ALATA_L',
 'INDOLEt2pp',
 'GLYt2pp_copy1',
 'HACD8i',
 'DHNPA2r_copy2',
 'TRPS1',
 'ACACT10',
 'CA2t3pp',
 'KAS15',
 'GLDBRAN2',
 'DHORD2',
 'KAS14',
 'ICHORS_copy2',
 'URAt2pp_copy1',
 'HACD2',
 'CTECOAI6_copy2',
 'HACD7i',
 'ACACT12',
 'CTECOAI8_copy2',
 'VPAMTr_copy2',
 'HACD3i',
 'HACD1',
 'ACACT11',
 'L_LACtex_copy2',
 'EAR121y',
 'ACACT8r_copy2',
 'GLCP',
 'GL

In [8]:
i= 0
r_id= diff_saturated_only_below_zero[i]
diff_saturatedFINAL_only_below_zero= list()
Ushaped_reactions_only_below_zero= list()

for r_id in diff_saturated_only_below_zero:   #separates saturated from U-shaped reactions below zero

    l= list()
    n=0
    n2=0
    r_id = diff_saturated_only_below_zero[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    
    if d.iloc[0,0] > d.iloc[1,0]-0.02 and d.iloc[0,0] < d.iloc[1,0]+0.02 or\
       d.iloc[1,0] > d.iloc[0,0]-0.02 and d.iloc[1,0] < d.iloc[0,0]+0.02 or\
       d.iloc[len(d)-2,0] > d.iloc[len(d)-1,0]-0.02 and d.iloc[len(d)-2,0] < d.iloc[len(d)-1,0]+0.02 or\
       d.iloc[len(d)-1,0] > d.iloc[len(d)-2,0]-0.02 and d.iloc[len(d)-1,0] < d.iloc[len(d)-2, 0]+0.02:
        diff_saturatedFINAL_only_below_zero.append(r_id)
    
    else:
        Ushaped_reactions_only_below_zero.append(r_id)
    
    i= i+1

diff_saturatedFINAL_only_below_zero
Ushaped_reactions_only_below_zero

['NTRIR4pp']

In [9]:
i= 0
r_id= diff_saturated[i]
diff_saturatedFINAL= list()
Ushaped_reactions= list()

for r_id in diff_saturated:

    l= list()
    n=0
    n2=0
    r_id = diff_saturated[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    
    if d.iloc[0,0] > d.iloc[1,0]-0.02 and d.iloc[0,0] < d.iloc[1,0]+0.02 or\
       d.iloc[1,0] > d.iloc[0,0]-0.02 and d.iloc[1,0] < d.iloc[0,0]+0.02 or\
       d.iloc[len(d)-2,0] > d.iloc[len(d)-1,0]-0.02 and d.iloc[len(d)-2,0] < d.iloc[len(d)-1,0]+0.02 or\
       d.iloc[len(d)-1,0] > d.iloc[len(d)-2,0]-0.02 and d.iloc[len(d)-1,0] < d.iloc[len(d)-2, 0]+0.02:
        diff_saturatedFINAL.append(r_id)
    
    else:
        Ushaped_reactions.append(r_id)
    
    i= i+1

set1= set(diff_saturatedFINAL)
set2= set(diff_saturatedFINAL_only_below_zero)

set3= set(Ushaped_reactions)
set4= set(Ushaped_reactions_only_below_zero)

diff_saturatedFINAL_both_positive_and_negative= list(set1-set2)   #finds saturated reactions for both above and below 0 flux values
Ushaped_reactions_both_positive_and_negative= list(set3-set4)     #fines U-shaped reactions for both above and below 0 flux values

diff_saturatedFINAL_both_positive_and_negative
Ushaped_reactions_both_positive_and_negative

['FEROpp',
 'ASNS1',
 'NTD8',
 'Htex',
 'GLYCK2',
 'EX_lys__L_e',
 'DHAPT',
 'GTPDPDP',
 'EX_h_e',
 'EX_fe3_e']

In [10]:
i2= 0
r_id = diff_indecr_only_above_zero[0]
mon_incr_only_above_zero= list()
mon_decr_only_above_zero= list()

for r_id in diff_indecr_only_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_only_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_only_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_only_above_zero.append(r_id)

    
    i2= i2 + 1

mon_decr_only_above_zero


['DM_oxam_c',
 'EX_12ppd__R_e',
 'EX_15dap_e',
 'EX_h2s_e',
 'EX_3hpp_e',
 'EX_his__L_e',
 'EX_hom__L_e',
 'EX_ile__L_e',
 'EX_acser_e',
 'EX_leu__L_e',
 'EX_cit_e',
 'EX_phe__L_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 'EX_spmd_e',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 'EX_thym_e',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_tyr__L_e',
 'EX_urea_e',
 '2MAHMP',
 '3HAD100',
 '3AMACHYD',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 '3HAD40',
 '3HAD60',
 '3HAD80',
 'AACPS9',
 'AACTOOR',
 '3HPPtpp',
 'ABTA',
 'ABUTD',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAR40',
 '3OAR60',
 '3OAR80',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS14

In [11]:
i2= 0
r_id = diff_indecr_only_below_zero[0]
mon_incr_only_below_zero= list()
mon_decr_only_below_zero= list()

for r_id in diff_indecr_only_below_zero:                 #this code separates reactions with an increasing trend from reactions with a decreasing trend below zero
    
    i=0
    r_id = diff_indecr_only_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_only_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_only_below_zero.append(r_id)

    
    i2= i2 + 1

mon_decr_only_below_zero


['EX_cmp_e',
 'EX_csn_e',
 'EX_glcur_e',
 'EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_23camp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_2ddglcn_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_cytd_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_acgam1p_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_mal__D_e',
 'EX_dgmp_e',
 'EX_man6p_e',
 'EX_amp_e',
 'EX_dgsn_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dtmp_e',
 'EX_dump_e',
 'EX_duri_e',
 'EX_psclys_e',
 'EX_pser__L_e',
 'EX_r5p_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_f6p_e',
 'EX_fald_e',
 'EX_thrp_e',
 'EX_thymd_e',
 'EX_trp__L_e',
 'EX_ump_e',
 'EX_uri_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g3pc_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLYBabcpp',
 'I

In [12]:
i2= 0
r_id = diff_indecr[0]
mon_incr= list()
mon_decr= list()

for r_id in diff_indecr:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend
    
    i=0
    r_id = diff_indecr[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr.append(r_id)

    
    i2= i2 + 1

set1= set(mon_decr)
set2= set(mon_decr_only_above_zero)
set3= set(mon_decr_only_below_zero)
mon_decr_both_positive_and_negative= list(set1-set2-set3)

set1= set(mon_incr)
set2= set(mon_incr_only_above_zero)
set3= set(mon_incr_only_below_zero)
mon_incr_both_positive_and_negative= list(set1-set2-set3)

mon_decr_both_positive_and_negative    #finds reactions with decreasing trend for both positive and negative fluz values
mon_incr_both_positive_and_negative    #finds reactions with increasing trend for both positive and negative fluz values

['GLCptspp', 'CRNDt2rpp', 'DURAD', 'LACZ', 'PGMT', 'GALt2pp']

In [13]:
i2=0
mon_decr_and_stable_only_above_zero= list()
mon_decr_only_only_above_zero= list()

while i2< len(mon_decr_only_above_zero):  #this code separates reactions that may have flat points from reactions that always decrease above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_decr_only_above_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable_only_above_zero.append(mon_decr_only_above_zero[i2])
    
    if boolean == False:
        mon_decr_only_only_above_zero.append(mon_decr_only_above_zero[i2])
    
    i2= i2+1

mon_decr_and_stable_only_above_zero

['FBA3', 'FRD3', 'LCTStpp']

In [14]:
i2=0
mon_decr_and_stable_only_below_zero= list()
mon_decr_only_only_below_zero= list()

while i2< len(mon_decr_only_below_zero):  #this code separates reactions that may have flat points from reactions that always decrease below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_decr_only_below_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable_only_below_zero.append(mon_decr_only_below_zero[i2])
    
    if boolean == False:
        mon_decr_only_only_below_zero.append(mon_decr_only_below_zero[i2])
    
    i2= i2+1

mon_decr_and_stable_only_below_zero

['EX_csn_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_acgam1p_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLYBabcpp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R2bpp',
 'XYLabcpp',
 'DKMPPD']

In [15]:
i2=0
mon_decr_and_stable_both_positive_and_negative= list()
mon_decr_only_both_positive_and_negative= list()

while i2< len(mon_decr_both_positive_and_negative):  #this code separates reactions that may have flat points from reactions that always decrease for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_decr_both_positive_and_negative[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable_both_positive_and_negative.append(mon_decr_both_positive_and_negative[i2])
    
    if boolean == False:
        mon_decr_only_both_positive_and_negative.append(mon_decr_both_positive_and_negative[i2])
    
    i2= i2+1

mon_decr_and_stable_both_positive_and_negative

['LEUabcpp',
 'ILEabcpp',
 'EX_cys__L_e',
 'CMtpp',
 'ARGabcpp',
 'ADNUC',
 'NADH9',
 'EX_glcn_e',
 'CTBTabcpp',
 'MLTG5',
 'SUCCt2_3pp',
 'PPK',
 'ASNabcpp',
 'MG2uabcpp',
 'IDOND2',
 'NTPP5',
 'ALAALAD',
 'FA140ACPHi',
 'PROabcpp',
 'NTP3',
 'ZN2abcpp',
 'R5PP',
 'HPYRRy',
 'XANt2pp',
 'NTPTP2',
 'LCTSt3ipp',
 'MG2tpp',
 'MDH3',
 'FACOAL60t2pp',
 'ALATA_D2',
 'ATPM',
 'FTHFLi',
 'NTPP3',
 'RFAMPtpp',
 'URIH',
 'MALt2_3pp',
 'ASPt2_2pp',
 'ASPabcpp',
 'POX',
 'ADPT',
 'ARBabcpp',
 'MLTG3',
 'CYSabc2pp',
 'GTHPi',
 'LYSabcpp',
 'GLYC3Pabcpp',
 'THRA2',
 'GALabcpp',
 'NTP10',
 'PTRCabcpp',
 'NADPHQR4',
 'NOVBCNtpp',
 'VALabcpp',
 'NTP5',
 'MINCYCtpp',
 'NADPHQR3',
 'ALAabcpp',
 'OAADC',
 'GDPMNP',
 'MDH2',
 'CYTBDpp',
 'HEX1',
 'NI2abcpp',
 'CRNDabcpp',
 'NADH10',
 'TTRCYCtpp',
 'PPA2',
 'FA160ACPHi',
 'NTPP4',
 'ORNabcpp',
 'EX_glyclt_e',
 'FA141ACPHi',
 'GLCabcpp',
 'ME1',
 'G3PD5',
 'MN6PP',
 'NADPHQR2',
 'NADTRHD',
 'ALDD2y',
 'DARBabcpp',
 'GLUN',
 'NTPP8',
 'FE2t2pp',
 'CRNabcpp',

In [16]:
i2=0
mon_incr_and_stable_only_above_zero= list()
mon_incr_only_only_above_zero= list()

while i2< len(mon_incr_only_above_zero):  #this code separates reactions that may have flat points from reactions that always increase above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_incr_only_above_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable_only_above_zero.append(mon_incr_only_above_zero[i2])
    
    if boolean == False:
        mon_incr_only_only_above_zero.append(mon_incr_only_above_zero[i2])
    
    i2= i2+1

mon_incr_and_stable_only_above_zero

['SPODM']

In [17]:
i2=0
mon_incr_and_stable_only_below_zero= list()
mon_incr_only_only_below_zero= list()

while i2< len(mon_incr_only_below_zero):  #this code separates reactions that may have flat points from reactions that always increase below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_incr_only_below_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable_only_below_zero.append(mon_incr_only_below_zero[i2])
    
    if boolean == False:
        mon_incr_only_only_below_zero.append(mon_incr_only_below_zero[i2])
    
    i2= i2+1

mon_incr_and_stable_only_below_zero

['DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'DSBAO2',
 'FRUpts2pp',
 'HYD2pp',
 'HYD3pp',
 'NO3R1pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'MECDPDH2']

In [18]:
i2=0
mon_incr_and_stable= list()
mon_incr_only= list()

while i2< len(mon_incr):  #this code separates reactions that may have flat points from reactions that always increase
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_incr[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable.append(mon_incr[i2])
    
    if boolean == False:
        mon_incr_only.append(mon_incr[i2])
    
    i2= i2+1


set1= set(mon_incr_and_stable)
set2= set(mon_incr_and_stable_only_above_zero)
set3= set(mon_incr_and_stable_only_below_zero)
mon_incr_and_stable_both_positive_and_negative= list(set1-set2-set3)

set4= set(mon_incr_only)
set5= set(mon_incr_only_only_above_zero)
set6= set(mon_incr_only_only_below_zero)
mon_incr_only_both_positive_and_negative= list(set4-set5-set6)

mon_incr_and_stable_both_positive_and_negative   #finds increasing and stable reactions for both positive and negative flux values
mon_incr_only_both_positive_and_negative         #finds increasing only reactions for both positive and negative flux values

['PGMT']

In [19]:
i=0
a= list()
b= list()
c= list()
d= list()
e= list()
actually_increasing_and_stable_below_zero_and_increasing_above_zero= list()
actually_stable_below_zero_and_increasing_above_zero= list()
stable_below_zero_and_decreasing_and_stable_above_zero= list()

while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        a.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            actually_increasing_and_stable_below_zero_and_increasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            actually_stable_below_zero_and_increasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == True and boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile3['Biomass'].unique()) > 1:
            
            stable_below_zero_and_decreasing_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
            
        
    i=i+1

actually_increasing_and_stable_below_zero_and_increasing_above_zero

['LEUabcpp',
 'ILEabcpp',
 'EX_cys__L_e',
 'CMtpp',
 'ARGabcpp',
 'NADH9',
 'EX_glcn_e',
 'CTBTabcpp',
 'MLTG5',
 'SUCCt2_3pp',
 'PPK',
 'ASNabcpp',
 'MG2uabcpp',
 'IDOND2',
 'NTPP5',
 'ALAALAD',
 'PROabcpp',
 'NTP3',
 'ZN2abcpp',
 'HPYRRy',
 'XANt2pp',
 'LCTSt3ipp',
 'MG2tpp',
 'MDH3',
 'ATPM',
 'NTPP3',
 'RFAMPtpp',
 'MALt2_3pp',
 'ASPt2_2pp',
 'ASPabcpp',
 'POX',
 'ARBabcpp',
 'MLTG3',
 'CYSabc2pp',
 'GTHPi',
 'LYSabcpp',
 'GLYC3Pabcpp',
 'THRA2',
 'GALabcpp',
 'NTP10',
 'PTRCabcpp',
 'NADPHQR4',
 'NOVBCNtpp',
 'VALabcpp',
 'NTP5',
 'MINCYCtpp',
 'NADPHQR3',
 'ALAabcpp',
 'OAADC',
 'MDH2',
 'CYTBDpp',
 'NI2abcpp',
 'CRNDabcpp',
 'NADH10',
 'TTRCYCtpp',
 'PPA2',
 'NTPP4',
 'ORNabcpp',
 'EX_glyclt_e',
 'GLCabcpp',
 'ME1',
 'G3PD5',
 'NADPHQR2',
 'NADTRHD',
 'ALDD2y',
 'DARBabcpp',
 'GLUN',
 'NTPP8',
 'FE2t2pp',
 'CRNabcpp',
 'CYTBD2pp',
 'ASPt2_3pp',
 'F6PP',
 'THIORDXi',
 'MLTG4',
 'EX_alaala_e',
 'NAt3_1p5pp',
 'NTP1',
 'THRabcpp',
 'NAt3_2pp',
 'FE2abcpp',
 'GLUabcpp',
 'ARBt3ipp',

In [47]:
i=0
L=list()

while i < len(actually_increasing_and_stable_below_zero_and_increasing_above_zero)-1:
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(actually_increasing_and_stable_below_zero_and_increasing_above_zero[i]))
    n= 0
    l= list()
        
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    df2 = len(csvFile2.Biomass)-len(csvFile2.Biomass.drop_duplicates())
    
    
    if df2<13:
        L.append(actually_increasing_and_stable_below_zero_and_increasing_above_zero[i])
    
    i=i+1
L

['EX_glcn_e']

In [20]:
i=0
a= list()
b= list()
c= list()
d= list()
e= list()
actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero= list()
actually_decreasing_only_below_zero_and_stable_above_zero= list()

while i < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_incr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        a.append(mon_incr_and_stable_both_positive_and_negative[i])
  
    if boolean == False and boolean2 == False:
        b.append(mon_incr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_incr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile3['Biomass'].unique()) > 1:
            actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile3['Biomass'].unique()) == 1:
            actually_decreasing_only_below_zero_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[i])
        
    if boolean == True:
        d.append(mon_incr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_incr_and_stable_both_positive_and_negative[i])
        
    i=i+1

actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero

['DURAD']

In [21]:
set1= set(diff_indecr_only_above_zero)
set2= set(mon_decr_only_above_zero)
set3= set(mon_incr_only_above_zero)
inANDdecr_only_above_zero= list(set1-set2-set3)  #finds increasing and decreasing reactions above zero
inANDdecr_only_above_zero

set1= set(diff_indecr_only_below_zero)
set2= set(mon_decr_only_below_zero)
set3= set(mon_incr_only_below_zero)
inANDdecr_only_below_zero= list(set1-set2-set3)  #finds increasing and decreasing reactions below zero
inANDdecr_only_below_zero

set1= set(diff_indecr)
set2= set(diff_indecr_only_above_zero)
set3= set(diff_indecr_only_below_zero)
diff_indecr_both_positive_and_negative= list(set1-set2-set3)

set1= set(diff_indecr_both_positive_and_negative)
set2= set(mon_decr_both_positive_and_negative)
set3= set(mon_incr_both_positive_and_negative)
inANDdecr_both_positive_and_negative= list(set1-set2-set3)   #finds increasing and decreasing reactions for both positive and negative flux values
inANDdecr_both_positive_and_negative

['4PCP',
 'NTD9',
 'PPCK',
 'COBALT2t3pp',
 'CD2t3pp',
 'G3PEabcpp',
 'Kabcpp',
 'EX_glyc__R_e',
 'GLCt2pp',
 'NMNN',
 'GLYCTO3',
 'GLYAT',
 'AKGDH',
 'GLYCTO4',
 'LPLIPAL2G181',
 'LYSt2pp',
 'PROD2',
 'MACPD',
 'CITL',
 'SUCDi',
 'NTD6',
 'SPMDabcpp',
 'FRD2',
 '2AGPGAT120',
 'CO2tpp',
 'AGM4PCP',
 'GPDDA4pp',
 'GSNK',
 'PYK',
 'DADK',
 'FORt2pp',
 'MOX',
 'MDDCP1pp',
 'ASNS2',
 'CYTDK2',
 'FEOXAMR1',
 'AGM3Pt2pp',
 'XYLI2',
 'EX_glc__D_e',
 'RNDR3',
 'LPLIPAL2E180',
 'DURIPP',
 'CYSDS',
 'FE3HOXR1',
 'MNt2pp',
 'MTHFD',
 'RNDR1b',
 'EAR141x',
 'CPGNR2',
 'SUCCt2_2pp',
 'NI2uabcpp',
 'LPLIPAL2E141',
 'IMPD',
 'GNK',
 'URIK2',
 'NACODA',
 'AGM3PA',
 'EX_asn__L_e',
 'EX_h2o_e',
 'OBTFL',
 'ECOAH1',
 'RNDR1',
 'EX_quin_e',
 'NMNAT',
 'THRt4pp',
 '2AGPGAT140',
 'FECRMR2',
 'HEX7',
 'ARBTNR1',
 'EX_indole_e',
 'LPLIPAL2E140',
 'PGPP180',
 'FADRx',
 'ARBTNR3',
 'EX_gal_e',
 'ARGDCpp',
 'EAR40x',
 'TRPS3',
 'MGSA',
 'EX_etoh_e',
 'ACOATA',
 'NDPK5',
 'PGM',
 'EAR161x',
 'TKT1',
 'PFK_3',
 'R

In [22]:
i= 0
r_id = inANDdecr_both_positive_and_negative[i]
l4= list()
l5= list()
l6= list()
l7= list()

while i< len(inANDdecr_both_positive_and_negative):   #separates inANDdecr_both_positive_and_negative reactions that actually always increase/decrease from the ones that don't
    
    r_id = inANDdecr_both_positive_and_negative[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    

    l= list()
    n2=100

    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    my_series = csvFile2['Biomass'].squeeze()
        
    if my_series.is_monotonic_decreasing ==True:
        l4.append(r_id)    #l4 contains reactions that have decreasing biomass values for positive flux values
    elif my_series.is_monotonic ==True:
        l5.append(r_id)    #l5 contains reactions that have increasing biomass values for positive flux values
    
    i=i+1

l4

['4PCP',
 'NTD9',
 'PPCK',
 'COBALT2t3pp',
 'CD2t3pp',
 'G3PEabcpp',
 'Kabcpp',
 'EX_glyc__R_e',
 'GLCt2pp',
 'NMNN',
 'GLYCTO3',
 'GLYAT',
 'AKGDH',
 'GLYCTO4',
 'LPLIPAL2G181',
 'LYSt2pp',
 'PROD2',
 'MACPD',
 'CITL',
 'NTD6',
 'SPMDabcpp',
 'FRD2',
 '2AGPGAT120',
 'CO2tpp',
 'AGM4PCP',
 'GPDDA4pp',
 'GSNK',
 'PYK',
 'DADK',
 'FORt2pp',
 'MOX',
 'MDDCP1pp',
 'ASNS2',
 'CYTDK2',
 'FEOXAMR1',
 'AGM3Pt2pp',
 'XYLI2',
 'EX_glc__D_e',
 'RNDR3',
 'LPLIPAL2E180',
 'DURIPP',
 'CYSDS',
 'FE3HOXR1',
 'MNt2pp',
 'MTHFD',
 'RNDR1b',
 'EAR141x',
 'CPGNR2',
 'SUCCt2_2pp',
 'NI2uabcpp',
 'LPLIPAL2E141',
 'IMPD',
 'GNK',
 'URIK2',
 'NACODA',
 'AGM3PA',
 'EX_asn__L_e',
 'OBTFL',
 'ECOAH1',
 'RNDR1',
 'EX_quin_e',
 'NMNAT',
 'THRt4pp',
 '2AGPGAT140',
 'FECRMR2',
 'HEX7',
 'ARBTNR1',
 'EX_indole_e',
 'LPLIPAL2E140',
 'PGPP180',
 'FADRx',
 'ARBTNR3',
 'EX_gal_e',
 'ARGDCpp',
 'EAR40x',
 'TRPS3',
 'MGSA',
 'EX_etoh_e',
 'ACOATA',
 'NDPK5',
 'PGM',
 'EAR161x',
 'TKT1',
 'PFK_3',
 'RNTR2c2',
 'EAR180x',
 '

In [23]:
i= 0
r_id = inANDdecr_both_positive_and_negative[i]
l7= list()

while i< len(inANDdecr_both_positive_and_negative):   #separates inANDdecr_both_positive_and_negative reactions that actually always increase/decrease from the ones that don't
    
    r_id = inANDdecr_both_positive_and_negative[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        
    l= list()
    n2=0

    while n2 < 101:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    my_series = csvFile2['Biomass'].squeeze()
        
    if my_series.is_monotonic_decreasing ==True:
        l6.append(r_id)    #l6 contains reactions that have decreasing biomass values for negative flux values
    elif my_series.is_monotonic ==True:
        l7.append(r_id)    #l7 contains reactions that have increasing biomass values for negative flux values
    
    i=i+1

l7


['4PCP',
 'NTD9',
 'COBALT2t3pp',
 'CD2t3pp',
 'G3PEabcpp',
 'NMNN',
 'GLYCTO3',
 'GLYAT',
 'AKGDH',
 'GLYCTO4',
 'LPLIPAL2G181',
 'LYSt2pp',
 'PROD2',
 'MACPD',
 'SUCDi',
 'NTD6',
 'SPMDabcpp',
 'FRD2',
 '2AGPGAT120',
 'AGM4PCP',
 'GPDDA4pp',
 'GSNK',
 'PYK',
 'DADK',
 'FORt2pp',
 'MOX',
 'MDDCP1pp',
 'ASNS2',
 'CYTDK2',
 'FEOXAMR1',
 'AGM3Pt2pp',
 'XYLI2',
 'RNDR3',
 'LPLIPAL2E180',
 'DURIPP',
 'CYSDS',
 'FE3HOXR1',
 'MTHFD',
 'RNDR1b',
 'EAR141x',
 'CPGNR2',
 'SUCCt2_2pp',
 'LPLIPAL2E141',
 'IMPD',
 'GNK',
 'URIK2',
 'NACODA',
 'AGM3PA',
 'EX_h2o_e',
 'OBTFL',
 'ECOAH1',
 'RNDR1',
 'EX_quin_e',
 'NMNAT',
 'THRt4pp',
 '2AGPGAT140',
 'FECRMR2',
 'HEX7',
 'ARBTNR1',
 'EX_indole_e',
 'LPLIPAL2E140',
 'PGPP180',
 'FADRx',
 'ARBTNR3',
 'EAR40x',
 'TRPS3',
 'MGSA',
 'ACOATA',
 'NDPK5',
 'EAR161x',
 'TKT1',
 'PFK_3',
 'RNTR2c2',
 'CYTBO3_4pp',
 'EAR180x',
 'MLDCP1Bpp',
 'AACPS3',
 'EAR121x',
 'LPLIPAL2E160',
 'FDH5pp',
 'AACPS1',
 'CLPNH141pp',
 '2AGPEAT141',
 'AM4PCP',
 'HXCT',
 'UM3PL',
 

In [24]:
D1= pd.DataFrame (l4, columns = ['Reaction names'])
D2= pd.DataFrame (l7, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))

set1= set(inANDdecr_both_positive_and_negative)
set2= set(union)
intersect = pd.Series(np.intersect1d(b, c))

inANDdecr_both_positive_and_negative_left= set1-set2
inANDdecr_both_positive_and_negative_left


set()

In [25]:
set3= set(l4)
set4= set(l7)

i=0
lnew=list()

while i<len(l4):
    if (l4[i] in l7) == False:
        lnew.append(l4[i])
    i=i+1

lnew

['PPCK',
 'Kabcpp',
 'EX_glyc__R_e',
 'GLCt2pp',
 'CITL',
 'CO2tpp',
 'EX_glc__D_e',
 'MNt2pp',
 'NI2uabcpp',
 'EX_asn__L_e',
 'EX_gal_e',
 'ARGDCpp',
 'EX_etoh_e',
 'PGM',
 'THD2pp',
 'EX_xan_e',
 'EX_12ppd__S_e',
 'FLDR2',
 'GLUDy',
 'FBP',
 'EX_alltn_e',
 'EX_gua_e',
 'ZNabcpp',
 'CAT',
 'GLYCDx',
 'H2Otex',
 'EX_glyc_e',
 'ME2',
 'H2Otpp',
 'CBPS',
 'PPA',
 'EX_idon__L_e',
 'EX_4abut_e',
 'PGK',
 'H2SO',
 'EX_ade_e',
 'EX_for_e',
 'PPS',
 'SULabcpp',
 'PIuabcpp',
 'EX_agm_e',
 'EX_ptrc_e',
 'EX_pro__L_e',
 'EX_hxa_e',
 'FE3abcpp',
 'EX_succ_e',
 'EX_ac_e',
 'EX_ala__D_e',
 'GLUSy',
 'CO2tex',
 'EX_cgly_e']

In [26]:
i=0
lnew2=list()

while i<len(l7):
    if (l7[i] in l4) == False:
        lnew2.append(l7[i])
    i=i+1

lnew2

['SUCDi',
 'EX_h2o_e',
 'CYTBO3_4pp',
 'PGL',
 'FUM',
 'EX_co2_e',
 'ENO',
 'TPI',
 'ATPS4rpp',
 'GND',
 'PGI',
 'GAPD',
 'MDH']

In [27]:
i2=0
inANDdecr_and_stable_only_above_zero= list()

while i2< len(inANDdecr_only_above_zero):  #this code finds increasing and decreasng and stable reactions above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_only_above_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr_only_above_zero[i2] not in inANDdecr_and_stable_only_above_zero:
            inANDdecr_and_stable_only_above_zero.append(inANDdecr_only_above_zero[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_only_above_zero


['O2tex', 'NH4tex']

In [28]:
i2=0
inANDdecr_and_stable_only_below_zero= list()

while i2< len(inANDdecr_only_below_zero):  #this code finds increasing and decreasng and stable reactions below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_only_below_zero[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr_only_below_zero[i2] not in inANDdecr_and_stable_only_below_zero:
            inANDdecr_and_stable_only_below_zero.append(inANDdecr_only_below_zero[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_only_below_zero

['EX_uacgam_e',
 'EX_xyl__D_e',
 'EX_xylu__L_e',
 'EX_udpgal_e',
 'EX_o2_e',
 'PPPGO',
 'OMMBLHX3',
 'OMPHHX3',
 'EX_udpg_e',
 'EX_rib__D_e',
 'EX_fru_e',
 'EX_arab__L_e',
 'EX_nh4_e',
 'EX_gam_e',
 'EX_g1p_e',
 'EX_cynt_e',
 'EX_h2_e',
 'EX_lyx__L_e',
 'NO3R1bpp',
 'OPHHX3',
 'EX_ppa_e',
 'NTRIR2x',
 'NTRIR3pp',
 'EX_sbt__D_e',
 'EX_galctn__D_e',
 'EX_ppal_e',
 'EX_mnl_e',
 'EX_galctn__L_e',
 'EX_manglyc_e',
 'EX_man_e']

In [29]:
i2=0
inANDdecr_and_stable_both_positive_and_negative= list()

while i2< len(inANDdecr_both_positive_and_negative):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_both_positive_and_negative[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr_both_positive_and_negative[i2] not in inANDdecr_and_stable_both_positive_and_negative:
            inANDdecr_and_stable_both_positive_and_negative.append(inANDdecr_both_positive_and_negative[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative


['NTD9',
 'PPCK',
 'Kabcpp',
 'EX_glyc__R_e',
 'GLYCTO3',
 'AKGDH',
 'GLYCTO4',
 'SUCDi',
 'FRD2',
 'CO2tpp',
 'GSNK',
 'PYK',
 'XYLI2',
 'EX_glc__D_e',
 'MNt2pp',
 'NI2uabcpp',
 'URIK2',
 'EX_h2o_e',
 'HEX7',
 'EX_gal_e',
 'ARGDCpp',
 'PFK_3',
 'FDH5pp',
 'HXCT',
 'FLDR2',
 'EX_gua_e',
 'ZNabcpp',
 'ASNN',
 'NADH18pp',
 'ACKr',
 'H2Otex',
 'EX_glyc_e',
 'CBPS',
 'NADH17pp',
 'PPA',
 'FBA',
 'H2SO',
 'GART',
 'INSK',
 'EX_for_e',
 'SULabcpp',
 'PDH',
 'PIuabcpp',
 'EX_agm_e',
 'ACALD',
 'TALA',
 'EX_ptrc_e',
 'DHORD5',
 'EX_pro__L_e',
 'NADH16pp',
 'PRPPS',
 'PTAr',
 'FE3abcpp',
 'PYRDC',
 'EX_ac_e',
 'ALATA_L2',
 'GLUSy',
 'ADNK1',
 'F6PA']

In [30]:
i2=0
inANDdecr_and_stable_both_positive_and_negative_intersect= list()

while i2< len(intersect):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(intersect[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and intersect[i2] not in inANDdecr_and_stable_both_positive_and_negative_intersect:
            inANDdecr_and_stable_both_positive_and_negative_intersect.append(intersect[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative_intersect

['ACALD',
 'ACKr',
 'ADNK1',
 'AKGDH',
 'ALATA_L2',
 'ASNN',
 'DHORD5',
 'F6PA',
 'FBA',
 'FDH5pp',
 'FRD2',
 'GART',
 'GLYCTO3',
 'GLYCTO4',
 'GSNK',
 'HEX7',
 'HXCT',
 'INSK',
 'NADH16pp',
 'NADH17pp',
 'NADH18pp',
 'NTD9',
 'PDH',
 'PFK_3',
 'PRPPS',
 'PTAr',
 'PYK',
 'PYRDC',
 'TALA',
 'URIK2',
 'XYLI2']

In [31]:
i2=0
inANDdecr_and_stable_both_positive_and_negative_lnew= list()

while i2< len(lnew):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(lnew[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and lnew[i2] not in inANDdecr_and_stable_both_positive_and_negative_lnew:
            inANDdecr_and_stable_both_positive_and_negative_lnew.append(lnew[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative_lnew

['PPCK',
 'Kabcpp',
 'EX_glyc__R_e',
 'CO2tpp',
 'EX_glc__D_e',
 'MNt2pp',
 'NI2uabcpp',
 'EX_gal_e',
 'ARGDCpp',
 'FLDR2',
 'EX_gua_e',
 'ZNabcpp',
 'H2Otex',
 'EX_glyc_e',
 'CBPS',
 'PPA',
 'H2SO',
 'EX_for_e',
 'SULabcpp',
 'PIuabcpp',
 'EX_agm_e',
 'EX_ptrc_e',
 'EX_pro__L_e',
 'FE3abcpp',
 'EX_ac_e',
 'GLUSy']

In [32]:
i2=0
inANDdecr_and_stable_both_positive_and_negative_lnew2= list()

while i2< len(lnew2):  #this code finds increasing and decreasng and stable reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(lnew2[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and lnew2[i2] not in inANDdecr_and_stable_both_positive_and_negative_lnew2:
            inANDdecr_and_stable_both_positive_and_negative_lnew2.append(lnew2[i2])
            
        i=i+1
        
    i2= i2+1

inANDdecr_and_stable_both_positive_and_negative_lnew2

['SUCDi', 'EX_h2o_e']

In [33]:
set1= set(inANDdecr_only_above_zero)
set2= set(inANDdecr_and_stable_only_above_zero)
l_left= list(set1-set2)
l_left

i2=0
inANDdecr_only_only_above_zero= list()

while i2< len(l_left):  #this code finds increasing and decreasng only reactions above zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in inANDdecr_only_only_above_zero:
            inANDdecr_only_only_above_zero.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

inANDdecr_only_only_above_zero

['AICART',
 'OMPDC',
 'ADSL2r',
 'GF6PTA',
 'UAMAGS',
 'THRS',
 'ARGSL',
 'GLUTRR',
 'ALAALAr',
 'UAMAS',
 'PRAIS',
 'ALAR',
 'AIRC2',
 'ACCOAC',
 'PRAGSr',
 'PMPK',
 'PSP_L',
 'ACGK',
 'PSD160',
 'ACONTb',
 'DHAD1',
 'PE161abcpp',
 'MCTP1App',
 'CTPS2',
 'ACPPAT160',
 'NDPK2',
 'ACODA',
 'NADK',
 '3OAR140',
 'BPNT',
 'GLUPRT',
 'ACLS',
 'PRASCSi',
 'UGMDDS',
 'GK1',
 'THRD_L',
 'ACONTa',
 'COBALT2tpp',
 'DTMPK',
 'SERAT',
 'O2tpp',
 'UAPGR',
 'ACPPAT161',
 'PRFGS',
 'HCO3E',
 'PSERT',
 'CS',
 'UAGCVT',
 'DASYN161',
 'G6PDH2r',
 'OCBT',
 '3OAS140',
 'SADT2',
 'HSK',
 'GMPS2',
 'CYSS',
 'NADS1',
 'PSD161',
 'UDCPDP',
 'MPTG',
 'ADSK',
 'ADSL1r',
 'DHFR',
 'NDPK4',
 'APG3PAT161',
 'SULR',
 'UAGPT3',
 'PAPPT3',
 'ASPCT',
 'PItex',
 'UAGDP',
 'PGCD',
 'TMPK',
 'NNATr',
 'SO4tex',
 'PSSA161',
 'CU2tpp',
 'UMPK',
 'APG3PAT160',
 'AGPAT161',
 'DASYN160',
 'ADSS',
 'ACGS',
 'ASPK',
 'GLUTRS',
 'UAAGDS',
 'G1PACT',
 'ICDHyr',
 'TMDS',
 'G1SAT',
 'PE160abcpp',
 'AGPAT160',
 'NH4tpp',
 'GLNS',
 '

In [34]:
set1= set(inANDdecr_only_below_zero)
set2= set(inANDdecr_and_stable_only_below_zero)
l_left= list(set1-set2)
l_left

i2=0
inANDdecr_only_only_below_zero= list()

while i2< len(l_left):  #this code finds increasing and decreasng only reactions below zero
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in inANDdecr_only_only_below_zero:
            inANDdecr_only_only_below_zero.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

inANDdecr_only_only_below_zero

['EX_so4_e',
 'RPI',
 'EX_ocdcea_e',
 'EX_LalaDgluMdapDala_e',
 'EX_ascb__L_e',
 'EX_acmum_e',
 'ASAD',
 'EX_acgam_e',
 'EX_hdca_e',
 'EX_anhgm_e',
 'GLUR',
 'ASPTA',
 'EX_dca_e',
 'AIRC3',
 'EX_udpglcur_e',
 'HSDy',
 'EX_tartr__L_e',
 'EX_galur_e',
 'EX_ddca_e',
 'EX_progly_e',
 'EX_hdcea_e',
 'EX_lcts_e',
 'EX_chtbs_e',
 'G3PD2',
 'ACOTA',
 'EX_fum_e',
 'EX_LalaDglu_e',
 'EX_gthrd_e',
 'EX_26dap__M_e',
 'EX_sucr_e',
 'EX_acmana_e',
 'EX_LalaLglu_e',
 'EX_3hcinnm_e',
 'EX_galt_e',
 'EX_octa_e',
 'EX_LalaDgluMdap_e',
 'KARA1',
 'EX_tartr__D_e',
 'EX_fe3dcit_e',
 'EX_tre_e',
 'EX_glcr_e',
 'EX_3hpppn_e',
 'EX_pppn_e',
 'EX_ttdca_e',
 'EX_pi_e',
 'AGPR',
 'EX_ocdca_e',
 'EX_malt_e',
 'EX_but_e',
 'EX_galct__D_e',
 'EX_all__D_e',
 'EX_ethso3_e',
 'PGAMT',
 'EX_ttdcea_e',
 'ORPT',
 'DHORTS',
 'EX_udpacgal_e',
 'EX_23ccmp_e',
 'EX_melib_e',
 'IMPC',
 'EX_23dappa_e',
 'EX_butso3_e',
 'EX_acnam_e']

In [35]:
set1= set(inANDdecr_both_positive_and_negative)
set2= set(inANDdecr_and_stable_both_positive_and_negative)
l_left= list(set1-set2)
l_left

i2=0
inANDdecr_only_both_positive_and_negative= list()

while i2< len(l_left):  #this code finds increasing and decreasng only reactions for both positive and negative flux values
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in inANDdecr_only_both_positive_and_negative:
            inANDdecr_only_both_positive_and_negative.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

inANDdecr_only_both_positive_and_negative


['4PCP',
 'COBALT2t3pp',
 'CD2t3pp',
 'G3PEabcpp',
 'GLCt2pp',
 'NMNN',
 'GLYAT',
 'LPLIPAL2G181',
 'LYSt2pp',
 'PROD2',
 'MACPD',
 'CITL',
 'NTD6',
 'SPMDabcpp',
 '2AGPGAT120',
 'AGM4PCP',
 'GPDDA4pp',
 'DADK',
 'FORt2pp',
 'MOX',
 'MDDCP1pp',
 'ASNS2',
 'CYTDK2',
 'FEOXAMR1',
 'AGM3Pt2pp',
 'RNDR3',
 'LPLIPAL2E180',
 'DURIPP',
 'CYSDS',
 'FE3HOXR1',
 'MTHFD',
 'RNDR1b',
 'EAR141x',
 'CPGNR2',
 'SUCCt2_2pp',
 'LPLIPAL2E141',
 'IMPD',
 'GNK',
 'NACODA',
 'AGM3PA',
 'EX_asn__L_e',
 'OBTFL',
 'ECOAH1',
 'RNDR1',
 'EX_quin_e',
 'NMNAT',
 'THRt4pp',
 '2AGPGAT140',
 'FECRMR2',
 'ARBTNR1',
 'EX_indole_e',
 'LPLIPAL2E140',
 'PGPP180',
 'FADRx',
 'ARBTNR3',
 'EAR40x',
 'TRPS3',
 'MGSA',
 'EX_etoh_e',
 'ACOATA',
 'NDPK5',
 'PGM',
 'EAR161x',
 'TKT1',
 'RNTR2c2',
 'CYTBO3_4pp',
 'EAR180x',
 'MLDCP1Bpp',
 'AACPS3',
 'EAR121x',
 'LPLIPAL2E160',
 'AACPS1',
 'THD2pp',
 'CLPNH141pp',
 '2AGPEAT141',
 'EX_xan_e',
 'AM4PCP',
 'UM3PL',
 'EX_12ppd__S_e',
 'GDPTPDP',
 'ARGAGMt7pp',
 'NTD1',
 'PGPP141',
 'G

In [36]:
i=0
truly_inANDdecr_only_both_positive_and_negative=list()

while i<len(inANDdecr_only_both_positive_and_negative):
    if (inANDdecr_only_both_positive_and_negative[i] in lnew) == True or (inANDdecr_only_both_positive_and_negative[i] in lnew2) == True:
        truly_inANDdecr_only_both_positive_and_negative.append(inANDdecr_only_both_positive_and_negative[i])
    i=i+1

truly_inANDdecr_only_both_positive_and_negative

['GLCt2pp',
 'CITL',
 'EX_asn__L_e',
 'EX_etoh_e',
 'PGM',
 'CYTBO3_4pp',
 'THD2pp',
 'EX_xan_e',
 'EX_12ppd__S_e',
 'GLUDy',
 'FBP',
 'EX_alltn_e',
 'CAT',
 'GLYCDx',
 'PGL',
 'ME2',
 'H2Otpp',
 'EX_idon__L_e',
 'FUM',
 'EX_4abut_e',
 'PGK',
 'EX_co2_e',
 'ENO',
 'TPI',
 'EX_ade_e',
 'PPS',
 'ATPS4rpp',
 'GND',
 'PGI',
 'EX_hxa_e',
 'GAPD',
 'MDH',
 'EX_succ_e',
 'EX_ala__D_e',
 'CO2tex',
 'EX_cgly_e']

In [39]:
"GLCt2pp" in l4

True

In [157]:
len(stable_below_zero_and_decreasing_and_stable_above_zero)

2

In [124]:
D= pd.DataFrame (one, columns = ['Reaction names'])
d1= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
d2= pd.DataFrame (flat_only_above_zero, columns = ['Reaction names'])
d3= pd.DataFrame (flat_only_below_zero, columns = ['Reaction names'])
d4= pd.DataFrame (flat_both_positive_and_negative, columns = ['Reaction names'])
d5= pd.DataFrame (diff_saturatedFINAL_only_below_zero, columns = ['Reaction names'])
d6= pd.DataFrame (Ushaped_reactions_only_below_zero, columns = ['Reaction names'])
d7= pd.DataFrame (diff_saturatedFINAL_both_positive_and_negative, columns = ['Reaction names'])
d8= pd.DataFrame (Ushaped_reactions_both_positive_and_negative, columns = ['Reaction names'])
d9= pd.DataFrame (mon_decr_and_stable_only_above_zero, columns = ['Reaction names'])
d10= pd.DataFrame (mon_decr_only_only_above_zero, columns = ['Reaction names'])
d11= pd.DataFrame (mon_decr_and_stable_only_below_zero, columns = ['Reaction names'])
d12= pd.DataFrame (mon_decr_only_only_below_zero, columns = ['Reaction names'])
d13= pd.DataFrame (mon_incr_and_stable_only_above_zero, columns = ['Reaction names'])
d14= pd.DataFrame (mon_incr_only_only_above_zero, columns = ['Reaction names'])
d15= pd.DataFrame (mon_incr_and_stable_only_below_zero, columns = ['Reaction names'])
d16= pd.DataFrame (mon_incr_only_only_below_zero, columns = ['Reaction names'])
d17= pd.DataFrame (mon_incr_only_both_positive_and_negative, columns = ['Reaction names'])
d18= pd.DataFrame (mon_decr_only_both_positive_and_negative, columns = ['Reaction names'])
d19= pd.DataFrame (actually_increasing_and_stable_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
d20= pd.DataFrame (actually_stable_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
d21= pd.DataFrame (actually_decreasing_only_below_zero_and_increasing_and_stable_above_zero, columns = ['Reaction names'])
d22= pd.DataFrame (actually_decreasing_only_below_zero_and_stable_above_zero, columns = ['Reaction names'])
d23= pd.DataFrame (inANDdecr_and_stable_only_above_zero, columns = ['Reaction names'])
d24= pd.DataFrame (inANDdecr_and_stable_only_below_zero, columns = ['Reaction names'])
d25= pd.DataFrame (inANDdecr_and_stable_both_positive_and_negative_intersect, columns = ['Reaction names'])
d26= pd.DataFrame (inANDdecr_and_stable_both_positive_and_negative_lnew, columns = ['Reaction names'])
d27= pd.DataFrame (inANDdecr_and_stable_both_positive_and_negative_lnew2, columns = ['Reaction names'])
d28= pd.DataFrame (inANDdecr_only_only_above_zero, columns = ['Reaction names'])
d29= pd.DataFrame (inANDdecr_only_only_below_zero, columns = ['Reaction names'])
d30= pd.DataFrame (inANDdecr_only_both_positive_and_negative, columns = ['Reaction names'])
d31= pd.DataFrame (stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])

len(D)+len(d1)+len(d2)+len(d3)+len(d4)+len(d5)+len(d6)+len(d7)+len(d8)+len(d9)+len(d10)+len(d11)+len(d12)+len(d13)+len(d14)+len(d15)+len(d16)+len(d17)+len(d18)+len(d19)+len(d20)+len(d21)+len(d22)+len(d23)+len(d24)+len(d25)+len(d26)+len(d27)+len(d28)+len(d29)+len(d30)+len(d31)

2726

In [41]:
D.to_csv("one feasible value NOT at zero reactions lactose.csv")
d1.to_csv("one feasible value at zero reactions lactose.csv")
d2.to_csv("flat only above zero lactose.csv")
d3.to_csv("flat only below zero lactose.csv")
d4.to_csv("flat both positive and negative lactose.csv")
d5.to_csv("diff_saturated only below zero lactose.csv")
d6.to_csv("Ushaped_reactions only below zero lactose.csv")
d7.to_csv("diff_saturated both positive and negative lactose.csv")
d8.to_csv("Ushaped both positive and negative lactose.csv")
d9.to_csv("monotonically decreasing and stable only above zero reactions lactose.csv")
d10.to_csv("monotonically decreasing only only above zero reactions lactose.csv")
d11.to_csv("monotonically increasing and stable only below zero reactions lactose.csv")
d12.to_csv("monotonically increasing only only below zero reactions lactose.csv")
d13.to_csv("monotonically increasing and stable only above zero reactions lactose.csv")
d14.to_csv("monotonically increasing only only above zero reactions lactose.csv")
d15.to_csv("monotonically decreasing and stable only below zero reactions lactose.csv")
d16.to_csv("monotonically decreasing only only below zero reactions lactose.csv")
d17.to_csv("decreasing only below zero and increasing only above zero reactions lactose.csv")
d18.to_csv("increasing only below zero and decreasing only above zero reactions lactose.csv")
d19.to_csv("increasing and stable only below zero and increasing only above zero reactions lactose.csv")
d20.to_csv("stable only below zero and increasing only above zero reactions lactose.csv")
d21.to_csv("decreasing only below zero and increasing and stable only above zero reactions lactose.csv")
d22.to_csv("decreasing only below zero and stable only above zero reactions lactose.csv")
d23.to_csv("increasing and decreasing and stable only above zero reactions lactose.csv")
d24.to_csv("increasing and decreasing and stable only below zero reactions lactose.csv")
d25.to_csv("decreasing and stable both positive and negative reactions lactose.csv")
d26.to_csv("increasing orand decreasing or both and stable below zero and decreasing above zero lactose.csv")
d27.to_csv("decreasing below zero and increasing or decreasing or both and stable above zero lactose.csv")
d28.to_csv("increasing and decreasing only above zero lactose.csv")
d29.to_csv("increasing and decreasing only below zero lactose.csv")
d30.to_csv("increasing and decreasing both positive and negative lactose.csv")
d31.to_csv("stable below zero and decreasing and stable above zero lactose.csv")


In [81]:
len(inANDdecr_and_stable_both_positive_and_negative)

59

In [82]:
len(inANDdecr_and_stable_both_positive_and_negative_intersect)+len(inANDdecr_and_stable_both_positive_and_negative_lnew)+len(inANDdecr_and_stable_both_positive_and_negative_lnew2)

59